In [2]:
%%writefile data.json
{
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Hi there", "How are you", "Is anyone there?","Hey","Hola", "Hello", "Good day"],
            "responses": ["Hello, thanks for asking", "Good to see you again", "Hi there, how can I help?"],
            "context": [""]
        },
        {
            "tag": "goodbye",
            "patterns": ["Bye", "See you later", "Goodbye", "Nice chatting to you, bye", "Till next time"],
            "responses": ["See you!", "Have a nice day", "Bye! Come back again soon."],
            "context": [""]
        },
        {
            "tag": "thanks",
            "patterns": ["Thanks", "Thank you", "That's helpful", "Awesome, thanks", "Thanks for helping me"],
            "responses": ["Happy to help!", "Any time!", "My pleasure"],
            "context": [""]
        },
        {
            "tag": "noanswer",
            "patterns": [],
            "responses": ["Sorry, can't understand you", "Please give me more info", "Not sure I understand"],
            "context": [""]
        },
        {
            "tag": "historical",
            "patterns": ["I want to check historical data", "How is the data yesterday?", "check historical"],
            "responses": ["Which sector do you want to check? (A/B)", "Which sector? (A/B)", "Provide the sector (A/B)"],
            "context": ["check_historical_A", "check_historical_B"]
        },
        {
            "tag": "check_historical_A",
            "patterns": ["Historical for sector A", "sector A", "check sector A"],
            "responses": ["Checking historical data for Sector A"],
            "context": [""]
        },
        {
            "tag": "check_historical_B",
            "patterns": ["Historical for sector B", "sector B", "check sector B"],
            "responses": ["Checking historical data for Sector B"],
            "context": [""]
        },
        {
            "tag": "current_data",
            "patterns": ["I want to check current data", "How is the data today?", "check current"],
            "responses": ["Which sector do you want to check? (A/B)", "Which sector? (A/B)", "Provide the sector (A/B)"],
            "context": ["check_current_data_A", "check_current_data_A"]
        },
        {
            "tag": "check_current_data_A",
            "patterns": ["Current data for sector A", "sector A", "check sector A"],
            "responses": ["Loading current data for Sector A"],
            "context": [""]
        },
        {
            "tag": "check_current_data_B",
            "patterns": ["Current data for sector B", "sector B", "check sector B"],
            "responses": ["Loading current data for Sector B"],
            "context": [""]
        },
        {
            "tag": "recommend_A",
            "patterns": ["Recommend for sector A", "what to do for sector A", "suggest for sector A"],
            "responses": ["Loading NPK status and recommend for sector A"],
            "context": [""]
        },
        {
            "tag": "recommend_B",
            "patterns": ["Recommend for sector B", "What to do for sector B", "suggest for sector B"],
            "responses": ["Loading NPK status and recommend for sector B"],
            "context": [""]
        },
        {
            "tag": "forecasting",
            "patterns": ["Forecast for tomorrow", "Please forecast", "forecast"],
            "responses": ["Which sector do you want to check? (A/B)", "Which sector? (A/B)", "Provide the sector (A/B)"],
            "context": ["do_forecasting_A", "do_forecasting_B"]
        },
        {
            "tag": "do_forecasting_A",
            "patterns": ["Forecast for sector A", "sector A", "forecast sector A"],
            "responses": ["Forecasting for Sector A"],
            "context": [""]
        },
        {
            "tag": "do_forecasting_B",
            "patterns": ["Forecast for sector B", "sector B", "forecast sector B"],
            "responses": ["Forecasting for Sector B"],
            "context": [""]
        }
    ]
}


Overwriting data.json


In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('data.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)

training

50 documents
14 classes ['check_current_data_A', 'check_current_data_B', 'check_historical_A', 'check_historical_B', 'current_data', 'do_forecasting_A', 'do_forecasting_B', 'forecasting', 'goodbye', 'greeting', 'historical', 'recommend_A', 'recommend_B', 'thanks']
51 unique lemmatized words ["'s", ',', 'a', 'anyone', 'are', 'awesome', 'b', 'bye', 'chatting', 'check', 'current', 'data', 'day', 'do', 'for', 'forecast', 'good', 'goodbye', 'hello', 'helpful', 'helping', 'hey', 'hi', 'historical', 'hola', 'how', 'i', 'is', 'later', 'me', 'next', 'nice', 'please', 'recommend', 'sector', 'see', 'suggest', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'today', 'tomorrow', 'want', 'what', 'yesterday', 'you']


[[[0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]],
 [[1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0

In [5]:
from keras.preprocessing.sequence import pad_sequences

# Pad sequences to ensure uniform length
max_seq_length = max(len(seq) for seq, _ in training)
train_x = pad_sequences([seq for seq, _ in training], maxlen=max_seq_length)

# Convert train_x back to a list of lists
train_x = train_x.tolist()

# Convert train_y to a numpy array
train_y = np.array([label for _, label in training])

print("Training data created")

# Now you can proceed with creating and compiling the model


Training data created


In [6]:
# training = np.array(training)
# # create train and test lists. X - patterns, Y - intents
# train_x = list(training[:,0])
# train_y = list(training[:,1])
# print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.07, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist)

print("model created")

Epoch 1/200
10/10 [==============================] - 1s 4ms/step - loss: 2.7067 - accuracy: 0.0600
Epoch 2/200
10/10 [==============================] - 0s 4ms/step - loss: 2.6238 - accuracy: 0.1000
Epoch 3/200
10/10 [==============================] - 0s 3ms/step - loss: 2.6281 - accuracy: 0.1200
Epoch 4/200
10/10 [==============================] - 0s 3ms/step - loss: 2.5521 - accuracy: 0.2200
Epoch 5/200
10/10 [==============================] - 0s 4ms/step - loss: 2.4202 - accuracy: 0.3200
Epoch 6/200
10/10 [==============================] - 0s 5ms/step - loss: 2.4180 - accuracy: 0.2400
Epoch 7/200
10/10 [==============================] - 0s 3ms/step - loss: 2.4488 - accuracy: 0.2400
Epoch 8/200
10/10 [==============================] - 0s 3ms/step - loss: 2.2817 - accuracy: 0.2200
Epoch 9/200
10/10 [==============================] - 0s 4ms/step - loss: 2.2333 - accuracy: 0.3000
Epoch 10/200
10/10 [==============================] - 0s 4ms/step - loss: 2.1079 - accuracy: 0.3800
Epoch 11/

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
import nltk
nltk.download('popular')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('model.h5')
import json
import random
intents = json.loads(open('data.json').read())
words = pickle.load(open('texts.pkl','rb'))
classes = pickle.load(open('labels.pkl','rb'))

def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res



[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

In [8]:
msg = "suggest sector A"

chatbot_response(msg)

1/1 [==============================] - 0s 391ms/step


'Loading NPK status and recommend for sector A'